In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from core.s3_utils import S3_Bucket
import pandas as pd
import re
import plotly.express as px

In [ ]:
from transform.raw_tss.main import get_raw_tss



# Load data from S3

In [ ]:
list_files = S3_Bucket().list_responses_keys_of_brand('stellantis')

In [ ]:
list_files

In [ ]:
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   #print(res[0][-3:])
   if res[0][-3:] == "720":
      print(file['key'])

In [ ]:
vin_list = ["VR3UKZKWZPJ789947", "VR3UKZKW5RJ625386", "VR3UKZKWZPJ911720", "VR3UKZKWZPJ911696", "VR3UKZKWZPJ918307", "VR3UKZKWZPJ789946",
            "VR3UKZKWZPJ853581", "VR3UKZKWZPJ911694", "VR3UKZKW6RJ619807", "VR3UKZKWZPJ911695", "VR3UKZKWZPJ842705", "VR3UKZKWZPJ918303"]

In [ ]:
df_stellantis = pd.DataFrame()
engine_df = pd.DataFrame()
electricty_df = pd.DataFrame()
df_final = pd.DataFrame()
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   if len(res) > 0:
      if res[0] in vin_list[12:]:
         data = S3_Bucket().read_json_file(file['key'])
         if data:
            vin = data.get('vin')
            odometer = pd.DataFrame(data.get('odometer', '')).rename(columns={'value': 'odometer'}).sort_values('datetime')
            odometer['datetime'] = pd.to_datetime(odometer['datetime'])
            battery_state = pd.DataFrame(data.get('engine', {}).get("battery", '')).sort_values('datetime')
            battery_state['datetime'] = pd.to_datetime(battery_state['datetime'])
            electricity =  data.get('electricity', {}).get("level", '')
            engine = data.get('engine', {}).get('contact', {})
            df = pd.merge(battery_state, odometer, on='datetime')
            if len(electricity) > 0:
               electricity = pd.DataFrame(electricity).rename(columns={'percentage': "batterie_percentage"}).sort_values('datetime')
               charging = pd.DataFrame(data.get("electricity").get('charging')).sort_values('datetime')
               electricity['datetime'] = pd.to_datetime(electricity['datetime'])
               charging['datetime'] = pd.to_datetime(charging['datetime'])
               autonomy = pd.DataFrame(data.get("electricity").get('residualAutonomy')).rename(columns={'value':'estimated_km'})
               autonomy['datetime'] = pd.to_datetime(autonomy['datetime'])
               elec = pd.merge(electricity, charging, on='datetime', how='outer').merge(autonomy, on='datetime', how='outer')
               electricty_df = pd.concat((electricty_df,elec ))
               
            
            if len(engine) > 0:
               
               engine = pd.DataFrame(engine).rename(columns={'value': 'contact'}).sort_values('datetime')
               engine['datetime'] = pd.to_datetime(engine['datetime'])
               engine_df = pd.concat((engine_df,engine ))
            
            df = df.sort_values('odometer')
            df_stellantis = pd.concat((df_stellantis, df))
         df_stellantis.drop(columns=['resistance', 'capacity'], inplace=True)      
         df_stellantis.reset_index(drop=True, inplace=True)   
         df_vin_test = electricty_df.merge(df_stellantis, on='datetime', how='outer').sort_values('datetime').drop_duplicates()
         df_vin_test['vin'] = vin
      # if len(df_vin_test) > 0:
      #    df_final = pd.concat((df_final, df_vin_test))


In [ ]:
df_stellantis = pd.DataFrame()
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   if len(res) > 0:
      if res[0] in vin_list:
         data = S3_Bucket().read_json_file(file['key'])
         if data:
            vin = data.get('vin')
            odometer = pd.DataFrame(data.get('odometer', '')).rename(columns={'value': 'odometer'}).sort_values('datetime')[["datetime","odometer"]]
            odometer['datetime'] = pd.to_datetime(odometer['datetime'])
            electricity =  data.get('electricity', {}).get("level", '')
            engine = data.get('engine', {}).get('contact', {})
            temp = pd.DataFrame(data.get('externalTemperature')).rename(columns={'value': 'external_temp'}).sort_values('datetime')[['external_temp', 'datetime']]
            temp['datetime'] = pd.to_datetime(temp['datetime'])
            df = pd.merge_asof(odometer, temp, on='datetime')
            
            if len(electricity) > 0:
               electricity = pd.DataFrame(electricity).rename(columns={'percentage': "soc"}).sort_values('datetime')
               charging = pd.DataFrame(data.get("electricity").get('charging')).sort_values('datetime')
               electricity['datetime'] = pd.to_datetime(electricity['datetime'])
               autonomy = pd.DataFrame(data.get("electricity").get('residualAutonomy'))[['datetime','value']].rename(columns={'value':'estimated_km'}).sort_values('datetime')
               autonomy['datetime'] = pd.to_datetime(autonomy['datetime'])
               charging['datetime'] = pd.to_datetime(charging['datetime'])
               elec = pd.merge_asof(electricity, charging, on='datetime')
               elec = pd.merge_asof(elec, autonomy, on='datetime')
               df =  pd.merge_asof(df, elec, on='datetime')

            if len(engine) > 0:
               engine = pd.DataFrame(engine).rename(columns={'value': 'contact'}).sort_values('datetime')
               engine['datetime'] = pd.to_datetime(engine['datetime'])
               df = pd.merge_asof(df, engine, on='datetime')
            
            df['vin'] = vin
            df = df.sort_values('odometer')
            df_stellantis = pd.concat((df_stellantis, df))
     
df_stellantis.reset_index(drop=True, inplace=True)


In [ ]:
df_stellantis.shape

In [ ]:
df_stellantis.isna().sum()

In [ ]:
df_stellantis.dropna(inplace=True)

In [ ]:
date_start_df = pd.DataFrame([['VR3UKZKWZPJ789947',"7-11-2023"],
['VR3UKZKW5RJ625386',"3-4-2024"],
['VR3UKZKWZPJ911720',"5-2-2024"],
['VR3UKZKWZPJ911696',"11-3-2024"],
['VR3UKZKWZPJ918307',"13-2-2024"],
['VR3UKZKWZPJ789946',"13-11-2023"],
['VR3UKZKWZPJ853581',"13-12-2023"],
['VR3UKZKWZPJ911694',"16-02-2024"],
['VR3UKZKW6RJ619807',"22-03-2024"],
['VR3UKZKWZPJ911695',"27-02-2024"],
['VR3UKZKWZPJ842705',"29-11-2023"],
['VR3UKZKWZPJ918303',"30-01-2024"],], columns=['vin', 'date_start'])
date_start_df['date_start'] = pd.to_datetime(date_start_df['date_start'],  dayfirst=True)

df_stellantis = df_stellantis.merge(date_start_df, on='vin')

In [ ]:
set(vin_list) - set(df_stellantis.vin.unique())

# SoH computation

On suppose que l'autonomie est 400 mais ce n'est pas forcément vrai mais le nombre estimé de km est de + de 370 pour chacun des véhicules sauf VR3UKZKWZPJ918303
Et sur le site de PSA l'estimation du dernier 2008 est comprise entre 400 et 406 km ou 341-343 km. 

In [ ]:
df_stellantis[df_stellantis['soc']>98].groupby('vin')['estimated_km'].max()

In [ ]:
test = df_stellantis[(df_stellantis['remainingTime']==0) & (df_stellantis['vin']=="VR3UKZKW6RJ619807")].copy()

In [ ]:
test.columns

In [ ]:
px.scatter(df_stellantis.groupby(['vin', 'date_start'], as_index=False)[['soh', 'estimated_km', 'soc', 'odometer']].mean(), x='date_start', y='soh', hover_data={ 
                                                                    'estimated_km': True, 
                                                                    'soc':True,
                                                                    'odometer':True}, color='vin')


In [ ]:
test['estimated_range_by_soc'] = test['estimated_km'] / test['soc']
px.scatter(test, x='soc', y='estimated_range_by_soc')

In [ ]:
px.imshow(test[['soc', 'odometer', 'external_temp', 'estimated_range_by_soc',]].corr()[['estimated_range_by_soc']].sort_values(by='estimated_range_by_soc', ascending=False)
)

In [ ]:
test['soh'] = test.apply(lambda x: (((x['estimated_km']) + max((20 -x['external_temp']), 0 )) / (400*x['soc']/100)), axis=1)

In [ ]:
px.scatter(test, x='soc', y='soh', hover_data={'estimated_km': True, 'external_temp':True})

In [ ]:
px.scatter(test, x='external_temp', y='soh', hover_data={'soc': True, 'estimated_km': True})

In [ ]:
px.scatter(test, x='estimated_km', y='soh')

 Transformation des variables


In [ ]:
pca.explained_variance_ratio_ # risque de dep de estimated_km

In [ ]:
df_stellantis = pd.concat((df_stellantis, data_pca), axis=1)

In [ ]:
df_stellantis.columns

In [ ]:
# compenser l'autonomie d'une température froide -> optimal suposé vers 20*C d'après internet
df_stellantis['soh'] = df_stellantis.apply(lambda x: ((x['estimated_km'])  / (400*x['soc']/100)), axis=1)
px.imshow(df_stellantis[ 
                        (df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')][['soh', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr())


### Première dépendance soc à étudier

In [ ]:
#df_stellantis['soh_temp'] = df_stellantis.apply(lambda x: ((x['estimated_km'] + 20 - .27 *x['external_temp'])  / (400*x['soc']/100)), axis=1)
df_stellantis['soh_norm_soc'] = df_stellantis['soh'] / df_stellantis['soc']
# df_stellantis['soh_norm_range'] = df_stellantis['soh'] / df_stellantis['estimated_km']
# df_stellantis['soh_norm_range_soc'] = (df_stellantis['soh'] / (df_stellantis['soc'] + df_stellantis['estimated_km']))

In [ ]:
px.imshow(df_stellantis[ 
                        (df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')][['soh_norm_soc', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr())


### PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
df_stellantis[['estimated_km_sc', 'soc_sc']] = scaler.fit_transform(df_stellantis[['estimated_km', 'soc']])
pca = PCA(n_components=1)
principal_components = pca.fit_transform(df_stellantis[['estimated_km_sc', 'soc_sc']])

# Création d'un nouveau DataFrame avec les nouvelles variables dés-corrélées
pca_columns = ['PC1_unique' ]
data_pca = pd.DataFrame(principal_components, columns=pca_columns)

df_stellantis['soh_pca'] = df_stellantis.apply(lambda x: ((x['estimated_km'] )  / (400*x['soc']/100)), axis=1)


In [ ]:
df_stellantis = pd.concat((df_stellantis, data_pca), axis=1)

In [ ]:
df_stellantis.columns

In [ ]:
df_stellantis['PC1_unique'].head(5)

In [ ]:
df_stellantis['soh_pca'] = df_stellantis.apply(lambda x: ((x['PC1bis'] )  / (400*x['soc'])), axis=1)

px.imshow(df_stellantis[ 
                        (df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')][['soh_pca', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr())


In [ ]:
px.scatter(df_stellantis[(df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')], y='soh_pca', x='soc', hover_data={'external_temp': True})

In [ ]:
px.scatter(df_stellantis[(df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')], y='soh_pca', x='estimated_km', hover_data={'external_temp': True})

In [ ]:
px.imshow(df_stellantis[ 
                        (df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')][['soh', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr())

In [ ]:
px.scatter(df_stellantis[(df_stellantis['plugged']==False) & (df_stellantis['status'] == 'disconnected')], 
           x='soc', y='soh', trendline='ols')

In [ ]:
px.scatter(df_stellantis[(df_stellantis['plugged']==False) & (df_stellantis['status'] == 'disconnected') & (df_stellantis['soc'] < 50)], 
           x='estimated_km', y='soh', trendline='ols')

In [ ]:
df_stellantis.groupby('vin').soh.mean()

In [ ]:
px.scatter(df_stellantis, x='odometer', y='soh', hover_data={'datetime':True, 
                                                                    'estimated_km': True, 
                                                                    'soc':True,
                                                                    'status': True,
                                                                    'plugged': True}, )


In [ ]:
px.scatter(df_stellantis[df_stellantis['plugged']==False], x='odometer', y='soh', hover_data={'datetime':True, 
                                                                    'estimated_km': True, 
                                                                    'soc':True,
                                                                    'status': True,
                                                                    "external_temp": True}, color='vin')


In [ ]:
px.scatter(df_stellantis.groupby(['vin', 'date_start'], as_index=False)[['soh', 'estimated_km', 'soc', 'odometer']].mean(), x='date_start', y='soh', hover_data={ 
                                                                    'estimated_km': True, 
                                                                    'soc':True,}, color='vin')


In [ ]:
df_stellantis[(df_stellantis['soc']>40) & (df_stellantis['plugged']==False)].groupby('vin').soh.mean()

In [ ]:
df_stellantis[df_stellantis['vin'] == vin_list[0]].soh.std()

In [ ]:
df_stellantis[df_stellantis['vin'] == vin_list[0]].soh.mean()

In [ ]:
df_stellantis[(df_stellantis['vin'] == vin_list[0]) & (df_stellantis['status'] == 'disconnected') &  (df_stellantis['contact'] == True)].soh.mean()

In [ ]:
import numpy as np

In [ ]:
0.9657274337347898 -  1.96 * (df_stellantis[(df_stellantis['vin'] == vin_list[0]) & (df_stellantis['status'] == 'disconnected') &  (df_stellantis['contact'] == True)].soh.std() / np.sqrt(len(df_stellantis[(df_stellantis['vin'] == vin_list[0]) & (df_stellantis['status'] == 'disconnected') &  (df_stellantis['contact'] == True)].soh)))

In [ ]:
(0.9657274337347898 +  1.96 * (df_stellantis[(df_stellantis['vin'] == vin_list[0]) & (df_stellantis['status'] == 'disconnected') &  (df_stellantis['contact'] == True)].soh.std() / np.sqrt(len(df_stellantis[(df_stellantis['vin'] == vin_list[0]) & (df_stellantis['status'] == 'disconnected') &  (df_stellantis['contact'] == True)].soh))))

In [ ]:
0.9657274337347898 + 1.96 * (df_stellantis[df_stellantis['vin'] == vin_list[0]].soh.std() / np.sqrt(len(df_stellantis[df_stellantis['vin'] == vin_list[0]])))

In [ ]:
df_stellantis[df_stellantis['vin'] == vin_list[0]].soh.median()